In [ ]:
import re
import os
import re
import json
import spacy
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
from tqdm import tqdm_notebook
from hdbcli import dbapi
import mysql.connector

# Get the Diagnosis Date for the Cohort with ICD&EF &Term in Clinical Notes
This Notebook is identifying the first Diagnois of Heart Failure (ICD Code or Clinical Note)

In [ ]:
#Load the needed MRNS
EF_Cohort = pq.read_table('Unique_HF_ICD_EF_Cohort.parquet').to_pandas()
mrns = EF_Cohort.index
mrns

In [ ]:
#load the all ICD diagnosis dates of the Cohort ICD & EF 
HF_Cohort=pq.read_table('ALL_HF_ICD_Cohort.parquet').to_pandas()
HF_Cohort=HF_Cohort[HF_Cohort['medical_record_number'].isin(mrns)]
HF_Cohort

In [ ]:
#load all CLinical Notes with HF related Term of the ICD EF Cohort 
Clinical_notes=pq.read_table('ALL_Matches_HF_ICD_Notes_Cohort.parquet').to_pandas()
Clinical_notes_HF_EF=Clinical_notes[Clinical_notes['MRN'].isin(mrns)]
Clinical_notes_HF_EF=Clinical_notes_HF_EF.sort_values(['MRN','age_in_days'], ascending=[False,True])
Clinical_notes_HF_EF

In [ ]:
def getNotesDates(mrn,age_in_days_icd) :
    #print(remaining_MRNS_notes)
    notes_specific_MRN= Clinical_notes_HF_EF.loc[Clinical_notes_HF_EF['MRN'] == int(mrn)]
    #print(notes_specific_MRN)
    for index,row_note in notes_specific_MRN.iterrows(): 
        age_in_days_note= row_note['age_in_days']
        if abs(age_in_days_note-age_in_days_icd)<=365: 
            #print(row_note)
            new_row  = {'MRN':row_note['MRN'],'age_in_days_icd': age_in_days_icd,'Note_ID':row_note['Note_ID'],'age_in_days': age_in_days_note, 'Term':row_note['Term']}
            #notes_ICD_EF_df_2= notes_ICD_EF_df_2.append(new_row, ignore_index=True)
            #print(notes_ICD_EF_df_2)
            return [True , new_row]
    return [False, '']

In [ ]:
def getICDDates(mrn,notes_ICD_EF_df):
    
    ICD_Dates= HF_Cohort.loc[HF_Cohort['medical_record_number'] == str(mrn)]
   # print(ICD_Dates)
    for index,row_icd in ICD_Dates.iterrows(): 
        age_in_days_icd= row_icd['age_in_days']
        #print(age_in_days_icd," Age")
        if Clinical_notes_HF_EF.loc[Clinical_notes_HF_EF['MRN'] == int(mrn)].empty==False:
            match=getNotesDates(mrn, age_in_days_icd)
            if match[0] == True:
                print(match[0])
                notes_ICD_EF_df= notes_ICD_EF_df.append(match[1], ignore_index=True)
                #print(notes_ICD_EF_df)
                return [True,notes_ICD_EF_df]
    return [False,'']

In [ ]:
notes_ICD_EF_df= pd.DataFrame(columns=['MRN','age_in_days_icd','Note_ID','age_in_days','Term'])
for index,row_icd in EF_Cohort.iterrows(): 
    mrn= index
    b=getICDDates(mrn,notes_ICD_EF_df)
    if b[0]==True:
        notes_ICD_EF_df=b[1]
    

In [ ]:
notes_ICD_EF_df

In [ ]:
notes_ICD_EF_df.to_parquet('ALL_Matches_1yr_HF_EF_ICD_Notes_Cohort.parquet')

In [ ]:
notes_ICD_EF_df=pq.read_table('ALL_Matches_100Days_HF_EF_ICD_Notes_Cohort.parquet').to_pandas()

In [ ]:
notes_ICD_EF_df

In [ ]:
#Add the column HF Onset  age_in_days and type
notes_ICD_EF_df["HF_Onset_age_in_days"]=""
notes_ICD_EF_df["HF_Onset_Type"]=""

In [ ]:
for index,row in notes_ICD_EF_df.iterrows(): 
    if row['age_in_days_icd']<row['age_in_days']:
        row['HF_Onset_age_in_days']=row['age_in_days_icd']
        row['HF_Onset_Type']='ICD'
    else: 
        row['HF_Onset_age_in_days']=row['age_in_days'] 
        row['HF_Onset_Type']='Note'

In [ ]:
notes_ICD_EF_df.to_parquet('ALL_Matches_1yr_HF_EF_ICD_Notes_Cohort.parquet')

# Get the Diagnosis Date for the Cohort with ICD &Term in Clinical Notes

In [ ]:
#Load already matches MRNS
EF_Cohort = pq.read_table('Unique_HF_ICD_EF_Cohort.parquet').to_pandas()
mrns = EF_Cohort.index
mrns

In [ ]:
#load the all ICD diagnosis dates of the Cohort ICD 
HF_Cohort=pq.read_table('ALL_HF_ICD_Cohort.parquet').to_pandas()
HF_Cohort=HF_Cohort[~HF_Cohort['medical_record_number'].isin(mrns)]
HF_Cohort

In [ ]:
#load all CLinical Notes with HF related Term of the ICD EF Cohort 
Clinical_notes=pq.read_table('ALL_Matches_HF_ICD_Notes_Cohort.parquet').to_pandas()
Clinical_notes_HF=Clinical_notes[~Clinical_notes['MRN'].isin(mrns)]
Clinical_notes_HF=Clinical_notes_HF.sort_values(['MRN','age_in_days'], ascending=[False,True])
Clinical_notes_HF

In [ ]:
#get list of MRNs for iterating over
Clinical_notes_HF_unique=Clinical_notes_HF.loc[~Clinical_notes_HF["MRN"].duplicated(keep='first')]
Clinical_notes_HF_unique

In [ ]:
def getNotesDates(mrn,age_in_days_icd) :
    #print(remaining_MRNS_notes)
    notes_specific_MRN= Clinical_notes_HF.loc[Clinical_notes_HF['MRN'] == int(mrn)]
    #print(notes_specific_MRN)
    for index,row_note in notes_specific_MRN.iterrows(): 
        age_in_days_note= row_note['age_in_days']
        if abs(age_in_days_note-age_in_days_icd)<=365: 
            #print(row_note)
            new_row  = {'MRN':row_note['MRN'],'age_in_days_icd': age_in_days_icd,'Note_ID':row_note['Note_ID'],'age_in_days': age_in_days_note, 'Term':row_note['Term']}
            #notes_ICD_EF_df_2= notes_ICD_EF_df_2.append(new_row, ignore_index=True)
            #print(notes_ICD_EF_df_2)
            return [True , new_row]
    return [False, '']

In [ ]:
def getICDDates(mrn,notes_ICD_df):
    
    ICD_Dates= HF_Cohort.loc[HF_Cohort['medical_record_number'] == str(mrn)]
   # print(ICD_Dates)
    for index,row_icd in ICD_Dates.iterrows(): 
        age_in_days_icd= row_icd['age_in_days']
        #print(age_in_days_icd," Age")
        if Clinical_notes_HF.loc[Clinical_notes_HF['MRN'] == int(mrn)].empty==False:
            match=getNotesDates(mrn, age_in_days_icd)
            if match[0] == True:
                print(match[0])
                notes_ICD_df= notes_ICD_df.append(match[1], ignore_index=True)
                #print(notes_ICD_df)
                return [True,notes_ICD_df]
    return [False,'']

In [ ]:
notes_ICD_df= pd.DataFrame(columns=['MRN','age_in_days_icd','Note_ID','age_in_days','Term'])
for index,row_icd in Clinical_notes_HF_unique.iterrows(): 
    mrn= row_icd['MRN']
    print(str(mrn))
    b=getICDDates(mrn,notes_ICD_df)
    if b[0]==True:
        notes_ICD_df=b[1]
    

In [ ]:
notes_ICD_df.to_parquet('ALL_Matches_1yr_HF_ICD_Notes_Cohort.parquet')

In [ ]:
notes_ICD_df

In [ ]:
#Add the column HF Onset  age_in_days and type
notes_ICD_df["HF_Onset_age_in_days"]=""
notes_ICD_df["HF_Onset_Type"]=""

In [ ]:
for index,row in notes_ICD_df.iterrows(): 
    if row['age_in_days_icd']<row['age_in_days']:
        row['HF_Onset_age_in_days']=row['age_in_days_icd']
        row['HF_Onset_Type']='ICD'
    else: 
        row['HF_Onset_age_in_days']=row['age_in_days'] 
        row['HF_Onset_Type']='Note'

In [ ]:
notes_ICD_df

In [ ]:
notes_ICD_df.to_parquet('ALL_Matches_1yr_HF_ICD_Notes_Cohort.parquet')